# Jupyter Notebook GenoRobotics Full Pipeline

## Imports

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import os.path as ospath
from lib.consensus.consensus import run_consensus
from lib.identification.identification import run_identification
from lib.general_helpers.process_fastq import concatenate_fastq
import os.path as ospath
from Bio import SeqIO

## Define Your File and Folder Paths

Modify wsl if you're on Windows and want to use WSL from windows (you are not running this code from wsl).

If the fastq file you want to choose is part of an expedition, you cna easily choose it by filling in the input expedition folder name and barcode

In [37]:
#for choosing a fastq that is part of an expedition easily
input_expedition_folder = "expedition_jardin_botanique"
barcode_nb = 88
windows = False

In [38]:
input_expedition_path= ospath.join("data", input_expedition_folder)

for root, dirs, files in os.walk(input_expedition_path):
    if root.endswith(str(f"barcode{barcode_nb}")):
        input_folder_path = root

for _,_, files in os.walk(input_folder_path):
    for file in files:
        if file.endswith(".fastq"):
            input_fastq_path = ospath.join(input_folder_path, file)
            input_fastq_filename=file
        if file.endswith(".fasta"):
            input_ref_path = ospath.join(input_folder_path, file)
        

print(f"pipeline will run on file {input_fastq_path}")
base_name = input_fastq_filename[:-6]

output_path = ospath.join("output",base_name)
print(f"results can be found in {output_path}")

pipeline will run on file data/expedition_jardin_botanique/Convallaria_magalis_trnH-psbA_barcode88/Convallaria_magalistrnH-psbA_barcode88.fastq
results can be found in output/Convallaria_magalistrnH-psbA_barcode88


If the fastq file you want to choose is not part of an expedition, you can directly fill in its name. **You should run either the previous two cells if it's part of an expedition, or the following cell if it's not, but never the three of them**

In [11]:
#for standalone fastq files outside any expedition
windows = False
input_fastq_file = "Convallaria_magalistrnH-psbA_barcode88.fastq"
input_expedition_folder = None

input_fastq_path = ospath.join("data", input_fastq_file)
base_name = ospath.splitext(input_fastq_filename)[0]
output_path = ospath.join("output",base_name)

## Quality Control

As a control, we check if the fastq file selected contains enough reads, aka. contains more reads than a certain threshold

In [12]:
threshold_fastq = 10

too_small = True
n=0
for read in SeqIO.parse(input_fastq_path, "fastq"):
    n+=1
    if n >= threshold_fastq:
        too_small = False
        break

if too_small:
    raise ValueError(f"The fastq file you selected contains {n} reads, less than the set threshold of {threshold_fastq} to run pipeline")

print("Passed control!")


Passed control!


## Run Consensus Sequence Generation

Select which consensus sequence generation method you want to use by setting the "consensus_method" variable to either:

- "majority" (default)

- "consensus"

- "consensus_with_ambiguities"

In [13]:
# choose a consensus method between the following:
# - "80_20_best_sequence"
# - "80_20_longest_sequence"
# - "straightforward_best_sequence"

# consensus_method = "straightforward_best_sequence"
consensus_method = "80_20_best_sequence"

run_consensus(input_name= base_name,
              expedition_name= input_expedition_folder,
              input_fastq_path= input_fastq_path, 
              consensus_method= consensus_method,
              windows= windows)

Logging set up at output/Convallaria_magalistrnH-psbA_barcode88/consensus/Convallaria_magalistrnH-psbA_barcode88_consensus_pipeline_log.log


(0.2067735195159912, 0.05065131187438965, 0.15612220764160156)

## Run Identification of Consensus Sequence
- Run the following cell to identify the consensus sequence.
- Change db to the database you want to use. Options are "matK", "rbcL", "psbA-trnH" and "ITS". If you want to use all of them, set db to None.

In [14]:
# choose an identification method between the following:
# - "blastn"

identification_method = "blastn"

# Choose your db along the genes you're trying to identify : matK, rbcL, psbA-trnH, ITS
db = {"psbA-trnH"}

run_identification(base_name,expedition_name=input_expedition_folder, db=db, identification_method=identification_method, windows=windows)

print("Pipeline finished !")
print(f"results can be found in {output_path}")

Logging set up at output/Convallaria_magalistrnH-psbA_barcode88/identification/Convallaria_magalistrnH-psbA_barcode88_identification_pipeline_log.log
{'psbA-trnH'}
Pipeline finished !
results can be found in output/Convallaria_magalistrnH-psbA_barcode88
